In [43]:
import pathlib
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, Float, String, Date, text
from sqlalchemy.orm import Session
from sqlalchemy import inspect, func

# Create path object using pathlib
This will ensure this file will run for everyone regardless of os
This group contains folks running on mac, windows, and linux.    

In [23]:
cwd = pathlib.Path.cwd()

if cwd.name == 'Project_3_Group_7':
    print("Path is project root")
else:
    print("Please correct current working directory to the project root")

Path is project root


In [24]:
resources_path = pathlib.PurePath(cwd, 'Resources')
resources_path

PurePosixPath('/home/mox/Documents/coding_projects/bootcamp_local/Homeworks/Project_3_Group_7/Resources')

# Create the db session
Can not use automap_base because primary key is a composite key 

https://python-code.dev/articles/132521261

In [25]:
engine = sqlalchemy.create_engine(f'sqlite:///{resources_path}/data.sqlite')
Base = declarative_base()

/tmp/ipykernel_27042/3378198577.py:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [26]:
inspector = inspect(engine)
tables = inspector.get_table_names()
tables

['steam_twitch_agg', 'tags', 'twitch_monthly']

In [27]:
steam_twitch_tags_table = tables[0]
twitch_monthly_table = tables[2]
tags_table = tables[1]

In [28]:
class steam_twitch_agg(Base):
    __tablename__ = steam_twitch_tags_table
    # https://stackoverflow.com/questions/19129289/how-to-define-composite-primary-key-in-sqlalchemy
    # indicates that this syntax should make a composite primary key
    app_id = Column(Integer, primary_key=True)
    tag = Column(String(128), primary_key=True)
    title = Column(String(128))
    # These two will need to be changed to Float 
    hours_watched = Column(Float)
    hours_streamed = Column(Float)
    average_streamers = Column(Integer)
    average_viewers = Column(Integer)
    average_channels = Column(Integer)
    release_date = Column(Date)
    reviews_total = Column(Integer)
    review_avg_percent = Column(Integer)
    launch_price_cents = Column(Integer)
    dataset_est_rev_cents = Column(Integer)

class twitch_monthly(Base):
    __tablename__ = twitch_monthly_table
    app_id = Column(Integer, primary_key=True)
    title = Column(String(128))
    rank = Column(Integer)
    month = Column(Integer, primary_key=True)
    year = Column(Integer, primary_key=True)
    hours_watched = Column(Integer)
    hours_streamed = Column(Integer)
    peak_viewers = Column(Integer)
    peak_channels = Column(Integer)
    streamers = Column(Integer)
    average_viewers = Column(Integer)
    average_channels = Column(Integer)

class steam_tags(Base):
    __tablename__ = tags_table
    app_id = Column(Integer, primary_key=True)
    tag = Column(String(128), primary_key=True)

In [29]:
Base.metadata.create_all(engine)

In [30]:
session = Session(bind= engine)

# Queries

In [31]:
helldivers_tags = ["Online Co-Op", "Third-Person Shooter", "PvE", "Multiplayer", "Action", "Shooter", "Co-op", "Sci-fi", "Capitalism", "Space", "Third Person", "Extraction Shooter", "Comedy", "Gore", "Violent", "Great Soundtrack", "Difficult", "Psychological Horror", "Singleplayer", "Open World"]

In [32]:
statement = text("SELECT app_id, title, avg_hours_streamed FROM steam_twitch_agg WHERE LOWER(title) LIKE 'hunt%' ORDER BY avg_hours_watched DESC;")
result = session.execute(statement).all()
hunt_S_app_id = result[0][0]
hunt_S_app_id

594650

In [33]:
statement = text("SELECT tag FROM tags WHERE app_id IS 594650;")
result = session.execute(statement).all()
hunt_tags = [val[0] for val in result]
hunt_tags

['Open World',
 'Tactical',
 'Atmospheric',
 'Multiplayer',
 'FPS',
 'Perma Death',
 'Shooter',
 'PvP',
 'Survival Horror',
 'Horror',
 'Difficult',
 'Online Co Op',
 'Co op',
 'Team Based',
 'Action',
 'Gore',
 'Zombies',
 'First Person']

In [34]:
shared_tags = [val for val in hunt_tags if val in helldivers_tags]
shared_tags

['Open World', 'Multiplayer', 'Shooter', 'Difficult', 'Action', 'Gore']

In [35]:
#inner_statement = text("SELECT app_id FROM tags WHERE tag IN ('Open World', 'Multiplayer', 'Shooter', 'Difficult', 'Action', 'Gore')")
statement = text(f"""SELECT st.*
FROM steam_twitch_agg AS st
JOIN tags as t
ON st.app_id = t.app_id
WHERE t.tag IN ('Open World', 'Multiplayer', 'Shooter', 'Difficult', 'Action', 'Gore')
GROUP BY st.app_id 
HAVING COUNT(st.app_id) > 3
ORDER BY st.avg_hours_streamed DESC;""")
result = session.execute(statement).all()
games_sharing_4_tags = result
print(len(games_sharing_4_tags))
games_sharing_4_tags

124


[(302, 271590, 'Grand Theft Auto V', 1123643, 1322782, 89.85, 64115728, 126038, 87926, 1539, '2015-04-13', 2999, 3967023218),
 (128, 730, 'Counter-Strike: Global Offensive', 906847, 7382695, 88.0, 48489362, 123006, 66604, 1243, '2012-08-21', 1499, 11066659805),
 (163, 381210, 'Dead by Daylight', 738369, 486959, 80.0, 13545274, 68595, 18559, 1011, '2016-06-14', 1999, 973431041),
 (174, 1085660, 'Destiny 2', 565819, 555551, 82.0, 7174193, 53869, 9832, 775, '2019-10-01', 3499, 1943872949),
 (517, 578080, 'PUBG: BATTLEGROUNDS', 495358, 2201296, 57.0, 9725767, 47808, 13337, 679, '2017-12-21', 2999, 6601686704),
 (213, 1245620, 'ELDEN RING', 334744, 520024, 92.0, 6275674, 48378, 8704, 463, '2022-02-24', 5999, 3119623976),
 (581, 1196590, 'Resident Evil Village', 294413, 64873, 95.0, 8070879, 34485, 10899, 397, '2021-05-06', 7999, 518919127),
 (571, 1282100, 'Remnant II', 283087, 6261, 81.0, 6976623, 32813, 9389, 380, '2023-07-25', 4999, 31298739),
 (48, 2208920, "Assassin's Creed Valhalla", 

In [36]:
#inner_statement = text("SELECT app_id FROM tags WHERE tag IN ('Open World', 'Multiplayer', 'Shooter', 'Difficult', 'Action', 'Gore')")
statement = text(f"""SELECT st.*
FROM steam_twitch_agg AS st
JOIN tags as t
ON st.app_id = t.app_id
WHERE t.tag IN ('Open World', 'Multiplayer', 'Shooter', 'Difficult', 'Action', 'Gore')
GROUP BY st.app_id 
HAVING COUNT(st.app_id) > 4
ORDER BY st.avg_hours_streamed DESC;""")
result = session.execute(statement).all()
games_sharing_5_tags = result
print(len(games_sharing_5_tags))
games_sharing_5_tags

28


[(569, 1174180, 'Red Dead Redemption 2', 236684, 394937, 90.0, 3104279, 30149, 4253, 324, '2019-12-05', 5999, 2369227063),
 (90, 397540, 'Borderlands 3', 128000, 93999, 85.0, 2058405, 14685, 2839, 175, '2020-03-13', 7999, 751898001),
 (352, 594650, 'Hunt: Showdown', 117086, 133530, 83.0, 2149006, 10008, 2959, 160, '2019-08-27', 3999, 533986470),
 (146, 782330, 'DOOM Eternal', 105214, 151201, 91.0, 2281359, 15405, 3086, 142, '2020-03-19', 5999, 907054799),
 (281, 1097840, 'Gears 5', 73735, 16677, 70.0, 919279, 8278, 1258, 100, '2019-09-09', 3999, 66691323),
 (576, 883710, 'Resident Evil 2', 71631, 92283, 97.0, 1442945, 12619, 1993, 98, '2019-01-24', 3999, 369039717),
 (561, 548570, 'RAGE 2', 66442, 12904, 62.0, 3004606, 10513, 4043, 89, '2019-05-14', 5999, 77411096),
 (428, 1030840, 'Mafia: Definitive Edition', 64473, 44916, 86.0, 3240675, 8852, 4482, 88, '2020-09-24', 3999, 179619084),
 (164, 895400, 'Deadside', 62758, 26380, 78.0, 1382600, 6331, 1922, 87, '2020-04-14', 1999, 52733620)

## Fall Guys is not the same genre as the rest. 
Initial statement used inner_statement and returned games that only matched single genre tags.
It matches some fo the tags but not all. How to match all tags...

In [37]:
# sqlalchemy solution https://stackoverflow.com/questions/13349832/sqlalchemy-filter-to-match-all-instead-of-any-values-in-list
# Used to build the SQL statements above
# results = session.query(steam_twitch_agg).\
#     join(steam_tags).\
#         filter(steam_tags.tag.in_(shared_tags)).\
#             group_by(steam_twitch_agg.app_id).\
#                 having(func.count(steam_twitch_agg.app_id) >= 4).all()

# Save dataframes for visualzation exploration

In [42]:
statement = text("PRAGMA table_info(steam_twitch_agg);")
result = session.execute(statement).all()
columns = [val[1] for val in result]
columns

['index',
 'app_id',
 'title',
 'avg_hours_streamed',
 'reviews_total',
 'review_avg_percent',
 'avg_hours_watched',
 'avg_streamers',
 'avg_viewers',
 'avg_channels',
 'release_date',
 'launch_price_cents',
 'dataset_est_rev_cents']

In [46]:
sharing_4_tags_df = pd.DataFrame(games_sharing_4_tags, columns=columns)
sharing_4_tags_df.head(3)

,index,app_id,title,avg_hours_streamed,reviews_total,review_avg_percent,avg_hours_watched,avg_streamers,avg_viewers,avg_channels,release_date,launch_price_cents,dataset_est_rev_cents
0,302,271590,Grand Theft Auto V,1123643,1322782,89.85,64115728,126038,87926,1539,2015-04-13,2999,3967023218
1,128,730,Counter-Strike: Global Offensive,906847,7382695,88.00,48489362,123006,66604,1243,2012-08-21,1499,11066659805
2,163,381210,Dead by Daylight,738369,486959,80.00,13545274,68595,18559,1011,2016-06-14,1999,973431041


In [45]:
sharing_5_tags_df = pd.DataFrame(games_sharing_5_tags, columns=columns)
sharing_5_tags_df.head(3)

,index,app_id,title,avg_hours_streamed,reviews_total,review_avg_percent,avg_hours_watched,avg_streamers,avg_viewers,avg_channels,release_date,launch_price_cents,dataset_est_rev_cents
0,569,1174180,Red Dead Redemption 2,236684,394937,90.0,3104279,30149,4253,324,2019-12-05,5999,2369227063
1,90,397540,Borderlands 3,128000,93999,85.0,2058405,14685,2839,175,2020-03-13,7999,751898001
2,352,594650,Hunt: Showdown,117086,133530,83.0,2149006,10008,2959,160,2019-08-27,3999,533986470


In [47]:
sharing_5_tags_df.to_csv(f'{resources_path}/Hunt-HD_gte5_tags_shared.csv', index=False)
sharing_4_tags_df.to_csv(f'{resources_path}/Hunt-HD_gte4_tags_shared.csv', index=False)

# Close session

In [48]:
session.close()